In [ ]:
import model as model
import math
import anchor as anchor
import random
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from nyu import my_dataloader as nyu_dataloader
from nyu import testingImageDir, center_test, test_lefttop_pixel, test_rightbottom_pixel, keypointsUVD_test, keypointsNumber
from nyu import cropWidth, cropHeight, depth_thres, xy_thres, downsample, keypointsNumber, net_stacks
from anchor import get_sparse_anchors
import cv2


test_image_datasets = nyu_dataloader(testingImageDir, center_test, test_lefttop_pixel, test_rightbottom_pixel, keypointsUVD_test, augment=False)

'''
joint_color = {
  0: '#CC0029',
  1: '#CC0029',
  2: '#FFFF00',
  3: '#FFFF00',
  4: '#008020',
  5: '#008020',
  6: '#007399',
  7: '#007399',
  8: '#CC9933',
  9: '#CC9933',
  10: '#CC9933',
  11: '#C7C7C7',
  12: '#C7C7C7',
  13: '#BF00FF',
}
'''
joint_color = {
  0: '#FF6600',
  1: '#FF3300',
  2: '#FFFF00',
  3: '#FFCC00',
  4: '#00FF00',
  5: '#00CC00',
  6: '#00FFFF',
  7: '#00CCFF',
  8: '#999900',
  9: '#CC9900',
  10: '#FF9900',
  11: '#DDDDDD',
  12: '#969696',
  13: '#BF00FF',
}

def view_cropimg():
    i = random.randint(0, len(test_image_datasets)-1)
    img, label = test_image_datasets[i]
    img, label = img.numpy(), label.numpy()
    img = img.squeeze(0)
    plt.plot(label[:,1], label[:,0], 'r*')
    plt.imshow(img)

def step_cut():
    i = random.randint(0, len(test_image_datasets)-1)
    img, label = test_image_datasets[i]
    img, label = img.numpy(), label.numpy()
    img = img.squeeze(0)
    mylabel = label.copy()
    mylabel[:, 0] = label[:, 1]
    mylabel[:, 1] = label[:, 0]
    label = mylabel

    plt.subplot(221)
    plt.plot(label[:,0], label[:,1], 'r*')
    plt.imshow(img)

    img, label = shape_transform(img, label)
    plt.subplot(222)
    plt.plot(label[:,0], label[:,1], 'r*')
    plt.imshow(img)

    plt.subplot(223)
    plt.plot(label[:,0], label[:,1], 'r*')
    plt.imshow(img)

def view_cropimg_with_anchor():
    batch_size = 1
    feature_size = int(cropHeight / downsample)

    i = random.randint(0, len(test_image_datasets)-1)
    img, label = test_image_datasets[i]
    joint_num, _ = label.shape
    post_process = anchor.MyProcess(cropHeight, keypointsNumber, downsample, net_stacks)
    net = model.A2J_model(num_classes = keypointsNumber, downsample=downsample, stack=net_stacks)
    net.load_state_dict(torch.load('result/model_store/50_stack4_34_8.31_sample.pth'))
    net = net.cuda()
    net.eval()
    heads = net(torch.unsqueeze(img.cuda(), dim=0))
    preds = post_process(heads)
    preds = preds.squeeze().cpu().detach().numpy()
    anchor_segmentation = post_process.get_anchor_segmentation(heads)
    anchor_segmentation = anchor_segmentation.squeeze().cpu().numpy()

    anchor_count = {i:0 for i in range(keypointsNumber)}
    for r in range(feature_size):
        for c in range(feature_size):
            anchor_count[anchor_segmentation[r, c]] += 1
    label_dis = {}
    for i in range(keypointsNumber):
        label_dis[i] = math.sqrt((label[i,0] - preds[i,0])**2 + (label[i,1] - preds[i,1])**2)
        print(i, label_dis[i], anchor_count[i])

    img = img.squeeze(0)
    for i in range(joint_num):
        plt.plot(label[i,1], label[i,0], '*', color=joint_color[i])
        plt.plot(preds[i,1], preds[i,0], '+', color=joint_color[i])
    plt.imshow(img)

    for r in range(feature_size):
        for c in range(feature_size):
            val = anchor_segmentation[r, c]
            plt.plot(c*downsample + downsample//2, r*downsample + downsample//2, 'o', color=joint_color[val])
    


In [ ]:
view_cropimg_with_anchor()

In [ ]:
import model as model
from nyu import keypointsNumber, downsample
net = model.A2J_model(num_classes = keypointsNumber, downsample=downsample)
net.load_state_dict(torch.load('result/model_store/1.pth'))
net = net.cuda()
net.eval()
post_precess = anchor.MyProcess(cropHeight, keypointsNumber, downsample)

In [ ]:
img, label = test_image_datasets[2145]
img, label = img.cuda(), label.cuda()
img, label = img.unsqueeze(0), label.unsqueeze(0)
output = net(img)
label_pred = post_precess(output)
label_pred = label_pred.squeeze(0)

img = img.squeeze().cpu().detach().numpy()
label = label.squeeze().cpu().detach().numpy()
label_pred = label_pred.squeeze().cpu().detach().numpy()

plt.plot(label[:,1], label[:,0], 'r*')
plt.plot(label_pred[:,1], label_pred[:,0], 'bo')
plt.imshow(img)